Here's how to do aggregations in MongoDB

In [13]:
!pip install pymongo

In [14]:
import pymongo
from pymongo import MongoClient

In [15]:
client = pymongo.MongoClient("mongodb+srv://<USER>:<PASSWORD>@<CLUSTERNAME>?retryWrites=true&w=majority")
db = client.environment
collection = db.noise_mapping

In [16]:
# Connect to local mongodb host
cursor = collection.aggregate([
  {
    "$match": {"Location/Agglomeration":"Milton Keynes Urban Area"}
  },
  {
    "$project" :  {
      "Location/Agglomeration": 1,
      "Road_Pop_Lden>=70dB": 1,
      "Railways_Pop_Lden>=70dB": 1,
      "Affects_More_People?":
        {
          "$switch":
            {
              "branches":[
                {
                    "case": {"$gt" : ["$Road_Pop_Lden>=70dB", "$Railways_Pop_Lden>=70dB"]}, "then": "Road"
                },
                {
                    "case": {"$gt" : ["$Railways_Pop_Lden>=70dB", "$Road_Pop_Lden>=70dB"]}, "then": "Rail"
                },
               ],
                "default": "No Difference"
            }
        }
    }
}])

In [17]:
list(cursor)

[{'_id': ObjectId('663a05011147d3732dd23e1e'),
  'Location/Agglomeration': 'Milton Keynes Urban Area',
  'Road_Pop_Lden>=70dB': 200,
  'Railways_Pop_Lden>=70dB': 600,
  'Affects_More_People?': 'Rail'}]